In [1]:
"""
Random Grids Pointwise Network Training for Google Colab
=========================================================
Complete script for training pointwise networks using Random Grids/Random Smiles
based on the approach by Baschetti et al.
"""

# ===============================================================
# === CELL 1: SETUP AND INSTALLATION ===
# ===============================================================
import os
from getpass import getpass

# 1. Enter token
token = getpass("Enter your GitHub PAT here (scope=repo): ")

# 2. Define project path and branch name
project_dir = "/content/deepLearningVolatility"
branch_name = "master"

# 3. Clean up residuals and clone the repository
if os.path.exists(project_dir):
    !rm -rf {project_dir}
!git clone --depth 1 --branch {branch_name} https://{token}@github.com/BianchiGiacomo/deepLearningVolatility.git {project_dir}

# 4. Install dependencies with Poetry
print("\n--> Installing project dependencies with Poetry...")
!cd {project_dir} && pip install poetry && poetry install

# 5. Verify installation
import importlib.util
spec = importlib.util.find_spec("deepLearningVolatility")

if spec:
    print("\n✓ Installation completed successfully!")
    print(f" The package 'deepLearningVolatility' is now installed.")
else:
    print("\nX ERROR: Installation seems to have failed.")

# Set working directory
os.chdir(project_dir)

Inserisci qui il tuo PAT GitHub (scope=repo): ··········
Cloning into '/content/deepLearningVolatility'...
remote: Enumerating objects: 125, done.
remote: Counting objects: 100% (125/125), done.
remote: Compressing objects: 100% (111/111), done.
remote: Total 125 (delta 18), reused 96 (delta 10), pack-reused 0 (from 0)
Receiving objects: 100% (125/125), 2.28 MiB | 15.09 MiB/s, done.
Resolving deltas: 100% (18/18), done.

--> Installing project dependencies with Poetry...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.7/278.7 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.6/332.6 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 453.8/453.8 kB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6

In [2]:
# ===============================================================
# === CELL 2: IMPORTS AND CONFIGURATION ===
# ===============================================================
import torch
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import pickle
import json
import gc
import time
from pathlib import Path
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import math

# Import from project libraries
from deepLearningVolatility.stochastic.stochastic_interface import ProcessFactory
from deepLearningVolatility.nn.dataset_builder import DatasetBuilder
from deepLearningVolatility.nn.pricer.pricer import PointwiseNetworkPricer

# Import wrappers to register them
import deepLearningVolatility.stochastic.wrappers.heston_wrapper
import deepLearningVolatility.stochastic.wrappers.rough_heston_wrapper
import deepLearningVolatility.stochastic.wrappers.rough_bergomi_wrapper

print("✔ Class imports successful!")
print("GPU available?", torch.cuda.is_available())
if torch.cuda.is_available():
    print("Device:", torch.cuda.get_device_name(0))

/content/deepLearningVolatility/deepLearningVolatility/nn/functional.py:291: SyntaxWarning: invalid escape sequence '\l'
  If :math:`p \leq 1 / N`` with :math:`N` being the number of elements to sort,
/content/deepLearningVolatility/deepLearningVolatility/nn/functional.py:332: SyntaxWarning: invalid escape sequence '\l'
  for :math:`\lambda\geq1`.


✔ Import delle classi riuscito!
GPU disponibile? True
Device: Tesla T4


In [3]:
# ===============================================================
# === CELL 3: CONFIGURATION AND GOOGLE DRIVE SETUP ===
# ===============================================================

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Main configuration
CONFIG = {
    # Stochastic process
    "process": "rough_bergomi",  # Options: "heston", "rough_heston", "rough_bergomi"
    "device": "cuda" if torch.cuda.is_available() else "cpu",

    # Dataset parameters - Random Grids
    "train_surfaces": 7_000,      # Number of surfaces for training
    "train_maturities": 11,       # Number of maturities per surface
    "train_strikes": 13,          # Number of strikes per maturity

    # Dataset parameters - Random Smiles for validation
    "val_smiles": 10_000,           # Number of smiles for validation
    "val_strikes": 13,            # Strikes per smile

    # Monte Carlo parameters
    "mc_paths": 50_000,            # Base number of MC paths
    "spot": 1.0,
    "seed": 42,

    # Training parameters
    "epochs": 100,
    "batch_size": 4096,
    "learning_rate": 1e-3,
    "lr_scheduler": True,
    "early_stopping_patience": 20,

    # Checkpoint settings
    "dataset_checkpoint_every": 500,  # Save every 100 surfaces
    "resume_dataset_generation": True,  # Resume from checkpoint if available
    "cleanup_checkpoints_after_completion": False,

    # Network architecture
    "hidden_layers": [30, 30, 30, 30],
    "activation": "ELU",

    # Training resume / warm-start
    "resume_training": True,                 # True to resume from checkpoint
    "resume_from": "latest",                  # "latest" or path to the .pt checkpoint
    "warm_start": True,                      # True to start from saved weights
    "warm_start_path": None,                  # if None uses best_model.pt or latest

    # Output directories
    "output_dir": "/content/drive/MyDrive/random_grids_results",
}

# Create directory structure on Drive
output_dir = CONFIG["output_dir"]
os.makedirs(f"{output_dir}/models", exist_ok=True)
os.makedirs(f"{output_dir}/models/checkpoints", exist_ok=True)
os.makedirs(f"{output_dir}/models/final", exist_ok=True)
os.makedirs(f"{output_dir}/datasets", exist_ok=True)
os.makedirs(f"{output_dir}/logs", exist_ok=True)
os.makedirs(f"{output_dir}/visualizations", exist_ok=True)

print(f"\n✔ Directories created in: {output_dir}")

Mounted at /content/drive

✔ Directory create in: /content/drive/MyDrive/random_grids_results


In [4]:
# ===============================================================
# === CELL 4: UTILITY FUNCTIONS ===
# ===============================================================

def print_dataset_stats(theta, T, k, iv, name="Dataset"):
    """Prints dataset statistics"""
    print(f"\n{'='*60}")
    print(f"{name} Statistics")
    print(f"{'='*60}")

    def _stats(x, label):
        return (f"{label:<10}: shape={str(tuple(x.shape)):<20} "
                f"mean={x.mean().item():.4f} std={x.std().item():.4f} "
                f"range=[{x.min().item():.4f}, {x.max().item():.4f}]")

    print(_stats(theta, "Theta"))
    print(_stats(T, "T"))
    print(_stats(k, "k"))
    print(_stats(iv, "IV"))
    print(f"Total points: {len(theta):,}")


def visualize_random_grids_sample(builder, theta, T, k, iv, n_samples=2):
    """Visualizes some examples of generated random grids"""
    fig, axes = plt.subplots(n_samples, 3, figsize=(15, 5*n_samples))

    if n_samples == 1:
        axes = axes.reshape(1, -1)

    # Denormalize for visualization
    theta_denorm = builder.denormalize_theta(theta)
    T_denorm = builder.denormalize_T(T) if builder.T_mean is not None else T
    k_denorm = builder.denormalize_k(k) if builder.k_mean is not None else k
    iv_denorm = builder.denormalize_iv(iv)

    for i in range(n_samples):
        # Find points for this theta
        theta_idx = i
        mask = torch.all(theta == theta[theta_idx], dim=1).cpu().numpy()

        T_sample = T_denorm[mask].cpu().numpy()
        k_sample = k_denorm[mask].cpu().numpy()
        iv_sample = iv_denorm[mask].cpu().numpy()

        # Scatter plot of the grid
        scatter = axes[i, 0].scatter(k_sample, T_sample, c=iv_sample,
                                     cmap='viridis', s=20)
        axes[i, 0].set_xlabel('Log-Moneyness')
        axes[i, 0].set_ylabel('Maturity')
        axes[i, 0].set_title(f'Sample {i+1}: Random Grid Points')
        plt.colorbar(scatter, ax=axes[i, 0], label='IV')

        # Distribution of maturities
        unique_T = np.unique(T_sample)
        axes[i, 1].hist(T_sample, bins=len(unique_T), alpha=0.7, edgecolor='black')
        axes[i, 1].set_xlabel('Maturity')
        axes[i, 1].set_ylabel('Count')
        axes[i, 1].set_title(f'Maturity Distribution ({len(unique_T)} unique)')

        # Approximate IV surface heatmap
        from scipy.interpolate import griddata
        Ti = np.linspace(T_sample.min(), T_sample.max(), 50)
        ki = np.linspace(k_sample.min(), k_sample.max(), 50)
        Ti_grid, ki_grid = np.meshgrid(Ti, ki)

        iv_interp = griddata((T_sample, k_sample), iv_sample,
                            (Ti_grid, ki_grid), method='linear')

        im = axes[i, 2].imshow(iv_interp.T, extent=[ki.min(), ki.max(),
                                                     Ti.min(), Ti.max()],
                               aspect='auto', origin='lower', cmap='viridis')
        axes[i, 2].set_xlabel('Log-Moneyness')
        axes[i, 2].set_ylabel('Maturity')
        axes[i, 2].set_title('Interpolated Surface')
        plt.colorbar(im, ax=axes[i, 2], label='IV')

    plt.tight_layout()
    plt.savefig(f"{CONFIG['output_dir']}/visualizations/random_grids_sample.png", dpi=150)
    plt.show()


def plot_training_history(train_losses, val_losses, save_path=None):
    """Visualizes training curves"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

    # Full loss
    ax1.plot(train_losses, label='Train Loss', linewidth=2)
    ax1.plot(val_losses, label='Val Loss', linewidth=2)
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('MSE Loss')
    ax1.set_title('Training History')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    ax1.set_yscale('log')

    # Zoom on last epochs
    zoom_start = max(0, len(train_losses) - 30)
    ax2.plot(range(zoom_start, len(train_losses)),
             train_losses[zoom_start:], label='Train Loss', linewidth=2)
    ax2.plot(range(zoom_start, len(val_losses)),
             val_losses[zoom_start:], label='Val Loss', linewidth=2)
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('MSE Loss')
    ax2.set_title('Last 30 Epochs')
    ax2.legend()
    ax2.grid(True, alpha=0.3)

    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, dpi=150)
    plt.show()

def cleanup_dataset_checkpoints(checkpoint_dir):
    """Removes dataset checkpoints after completion."""
    import shutil
    if os.path.exists(checkpoint_dir):
        print(f"Cleaning up checkpoints in {checkpoint_dir} ...")
        shutil.rmtree(checkpoint_dir)
        print("✓ Checkpoints removed")

def test_predictions(pricer, builder, theta_test, T_test, k_test, iv_test, n_samples=5):
    """Tests model predictions"""
    pricer.eval()

    with torch.no_grad():
        # Predictions (output already denormalized by price_iv)
        iv_pred = pricer.price_iv(theta_test, T_test, k_test, denormalize_output=True, inputs_normalized=True)

        # Denormalize true values
        iv_true = builder.denormalize_iv(iv_test)

        # Calculate metrics
        mae = (iv_pred - iv_true).abs().mean()
        rmse = ((iv_pred - iv_true) ** 2).mean().sqrt()
        mape = ((iv_pred - iv_true).abs() / (iv_true + 1e-8)).mean() * 100

        print(f"\nTest Metrics:")
        print(f"  MAE:  {mae:.6f}")
        print(f"  RMSE: {rmse:.6f}")
        print(f"  MAPE: {mape:.2f}%")

        # Visualize some examples
        fig, axes = plt.subplots(1, n_samples, figsize=(4*n_samples, 4))
        if n_samples == 1:
            axes = [axes]

        for i in range(min(n_samples, len(theta_test))):
            # Find all points for this theta
            theta_i = theta_test[i:i+1]
            mask = torch.all(theta_test == theta_i, dim=1)

            T_i = builder.denormalize_T(T_test[mask]) if builder.T_mean is not None else T_test[mask]
            k_i = builder.denormalize_k(k_test[mask]) if builder.k_mean is not None else k_test[mask]
            iv_true_i = iv_true[mask]
            iv_pred_i = iv_pred[mask]

            # Sort by T then k for visualization
            sort_idx = torch.argsort(T_i * 1000 + k_i)

            axes[i].scatter(k_i[sort_idx].cpu(), iv_true_i[sort_idx].cpu(),
                          alpha=0.6, label='True', s=20)
            axes[i].scatter(k_i[sort_idx].cpu(), iv_pred_i[sort_idx].cpu(),
                          alpha=0.6, label='Predicted', s=20)
            axes[i].set_xlabel('Log-Moneyness')
            axes[i].set_ylabel('IV')
            axes[i].set_title(f'Sample {i+1}')
            axes[i].legend()
            axes[i].grid(True, alpha=0.3)

        plt.tight_layout()
        plt.savefig(f"{CONFIG['output_dir']}/visualizations/predictions_comparison.png", dpi=150)
        plt.show()

    return mae, rmse, mape

def generate_mc_reference_surface(builder, theta, maturities, logK, n_paths=50000):
    """
    Generates reference surface via Monte Carlo for comparison

    Args:
        builder: DatasetBuilder with the process
        theta: Model parameters (not normalized)
        maturities: Array of maturities
        logK: Array of log-moneyness
        n_paths: Number of MC paths for high precision
    """
    print(f"Generating MC reference surface with {n_paths} paths...")

    # Create a temporary pricer for MC
    from deepLearningVolatility.nn.pricer.pricer import GridNetworkPricer

    maturities = maturities.to(builder.device).float().contiguous()
    logK = logK.to(builder.device).float().contiguous()
    theta = theta.to(builder.device).float().contiguous()

    temp_pricer = GridNetworkPricer(
        maturities=maturities,
        logK=logK,
        process=builder.process,
        device=builder.device,
        enable_smile_repair=True,
        smile_repair_method='pchip'
    )

    # Generate surface via MC
    with torch.no_grad():
        iv_mc = temp_pricer._mc_iv_grid(
            theta=theta,
            n_paths=n_paths,
            spot=1.0,
            use_antithetic=True,
            adaptive_dt=True,
            control_variate=True
        )

    return iv_mc

def visualize_mc_vs_nn_comparison(builder, pricer, theta, maturities, logK,
                                  n_mc_paths=50000, save_path=None):
    """
    Compares MC vs NN using, for each T, an **in-domain** k-grid (as in training).
    Note: `len(logK)` only determines the number of points per smile; k values
    are recalculated for each T based on √T.
    """
    n_T = len(maturities)
    n_K = len(logK)  # only the NUMBER of points per smile

    # Strike domain parameters used in training (fallback to defaults)
    sp = getattr(builder, "_strike_params", {"l": 0.55, "u": 0.30})
    l, u = float(sp["l"]), float(sp["u"])
    spot = 1.0
    safety_eps = 1e-6

    # Preallocate
    k_mat  = torch.empty((n_T, n_K), device=builder.device, dtype=torch.float32)
    iv_mc  = torch.empty((n_T, n_K), device=builder.device, dtype=torch.float32)
    iv_nn  = torch.empty((n_T, n_K), device=builder.device, dtype=torch.float32)

    # Per-T calculation (no extrapolation)
    pricer.eval()
    with torch.no_grad():
        for j, T in enumerate(maturities):
            T_val = float(T.item())
            sqrtT = T_val ** 0.5
            K_min = max(spot * (1.0 - l * sqrtT), 0.05 * spot)
            K_max = min(spot * (1.0 + u * sqrtT), 3.00 * spot)
            k_min = math.log(K_min / spot) + safety_eps
            k_max = math.log(K_max / spot) - safety_eps
            k_vec = torch.linspace(k_min, k_max, n_K, device=builder.device)
            k_mat[j, :] = k_vec

            # MC on (T, k_vec)
            iv_mc_j = pricer._mc_iv_grid(
                theta=theta,
                maturities=T.view(1).to(builder.device),
                logK=k_vec,
                n_paths=n_mc_paths,
                spot=spot,
                use_antithetic=True,
                control_variate=True
            ).squeeze(0)
            iv_mc[j, :] = iv_mc_j

            # NN on the same grid (RAW input → default inputs_normalized=False)
            iv_nn_j = pricer.price_iv_grid(
                theta=theta,
                maturities=T.view(1).to(builder.device),
                logK=k_vec,
                denormalize_output=True
            ).squeeze(0)
            iv_nn[j, :] = iv_nn_j


    # Figure for smile comparison (max 5 maturities)
    n_T_to_plot = min(5, n_T)
    fig, axes = plt.subplots(1, n_T_to_plot, figsize=(5 * n_T_to_plot, 5), squeeze=False)

    for col, T_idx in enumerate(range(n_T_to_plot)):
        ax = axes[0, col]
        T_val = maturities[T_idx].item()

        # Plot MC vs NN with the specific k-grid for this T
        ax.plot(k_mat[T_idx, :].cpu(), iv_mc[T_idx, :].cpu(), 'b-o',
                linewidth=2, label='True (MC)', markersize=5)
        ax.plot(k_mat[T_idx, :].cpu(), iv_nn[T_idx, :].cpu(), 'r--s',
                linewidth=2, label='Predicted (NN)', markersize=4)

        # Formatting
        ax.set_title(f'T = {T_val:.3f} years', fontsize=12)
        if col == 0:
            ax.set_ylabel('Implied Volatility', fontsize=10)
        ax.set_xlabel('Log-Moneyness (k)', fontsize=10)
        ax.grid(True, alpha=0.4)
        ax.legend(fontsize=9)

        # Calculate MAE for this smile
        mae = (iv_mc[T_idx, :] - iv_nn[T_idx, :]).abs().mean().item()
        ax.text(0.03, 0.97, f'MAE: {mae:.4f}',
                transform=ax.transAxes,
                verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.7),
                fontsize=8)

    # Add parameters as title
    param_names = builder.process.param_info.names if hasattr(builder.process, 'param_info') else []
    if param_names:
        param_str = ", ".join([f"{name}={val:.3f}" for name, val in zip(param_names, theta.cpu().numpy())])
        fig.suptitle(f'MC vs NN Smile Comparison\n{param_str}', fontsize=14)
    else:
        fig.suptitle('MC vs NN Smile Comparison', fontsize=14)

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])

    if save_path:
        plt.savefig(save_path, dpi=150)
    plt.show()

    # Calculate and print global statistics
    mae_global = (iv_mc - iv_nn).abs().mean().item()
    rmse_global = ((iv_mc - iv_nn) ** 2).mean().sqrt().item()
    max_error = (iv_mc - iv_nn).abs().max().item()

    print(f"\nGlobal Comparison Statistics:")
    print(f"  MAE:  {mae_global:.6f}")
    print(f"  RMSE: {rmse_global:.6f}")
    print(f"  Max Error: {max_error:.6f}")

    return iv_mc, iv_nn

def visualize_surface_heatmap_comparison(iv_mc, iv_nn, maturities, logK, save_path=None):
    """
    Visualizes surface comparison as heatmap (True, Predicted, Error)
    """
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))

    # True surface (MC)
    im0 = axes[0].imshow(iv_mc.cpu(), aspect='auto', cmap='viridis',
                         extent=[logK.min().item(), logK.max().item(),
                                maturities.min().item(), maturities.max().item()],
                         origin='lower')
    axes[0].set_title('True IV Surface (MC)')
    axes[0].set_xlabel('Log-Moneyness')
    axes[0].set_ylabel('Maturity')
    fig.colorbar(im0, ax=axes[0])

    # Predicted surface (NN)
    im1 = axes[1].imshow(iv_nn.cpu(), aspect='auto', cmap='viridis',
                         extent=[logK.min().item(), logK.max().item(),
                                maturities.min().item(), maturities.max().item()],
                         origin='lower')
    axes[1].set_title('Predicted IV Surface (NN)')
    axes[1].set_xlabel('Log-Moneyness')
    fig.colorbar(im1, ax=axes[1])

    # Error surface
    error = (iv_nn - iv_mc).abs()
    im2 = axes[2].imshow(error.cpu(), aspect='auto', cmap='hot',
                         extent=[logK.min().item(), logK.max().item(),
                                maturities.min().item(), maturities.max().item()],
                         origin='lower')
    axes[2].set_title(f'Absolute Error (max: {error.max():.4f})')
    axes[2].set_xlabel('Log-Moneyness')
    fig.colorbar(im2, ax=axes[2])

    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, dpi=150)
    plt.show()

In [6]:
# ===============================================================
# === CELL 5: DATASET GENERATION WITH RANDOM GRIDS ===
# ===============================================================

def generate_datasets():
    """Generates training and validation datasets using Random Grids/Smiles"""

    print("\n" + "="*60)
    print("DATASET GENERATION WITH RANDOM GRIDS")
    print("="*60)

    # Set seed for reproducibility
    np.random.seed(CONFIG["seed"])
    torch.manual_seed(CONFIG["seed"])

    # Create stochastic process
    process = ProcessFactory.create(CONFIG["process"])
    print(f"\n✔ Created process: {process.__class__.__name__}")
    print(f"  Parameters: {process.param_info.names if hasattr(process, 'param_info') else 'N/A'}")
    print(f"  Supports absorption: {process.supports_absorption}")

    # Initialize DatasetBuilder
    train_builder = DatasetBuilder(
        process=process,
        device=CONFIG["device"],
        output_dir=CONFIG["output_dir"],
        dataset_type='train'
    )

    # --- TRAINING DATASET with Random Grids ---
    print(f"\n>>> Generating Training Dataset with Random Grids")
    print(f"    Surfaces: {CONFIG['train_surfaces']}")
    print(f"    Maturities per surface: {CONFIG['train_maturities']}")
    print(f"    Strikes per maturity: {CONFIG['train_strikes']}")
    print(f"    Total points: {CONFIG['train_surfaces'] * CONFIG['train_maturities'] * CONFIG['train_strikes']:,}")

    t_start = time.time()

    theta_train, T_train, k_train, iv_train = train_builder.build_random_grids_dataset(
        n_surfaces=CONFIG["train_surfaces"],
        n_maturities=CONFIG["train_maturities"],
        n_strikes=CONFIG["train_strikes"],
        n_paths=CONFIG["mc_paths"],
        spot=CONFIG["spot"],
        normalize=True,
        compute_stats_from=None,
        show_progress=True,
        batch_size=50,
        checkpoint_every=CONFIG.get("dataset_checkpoint_every", 100),
        resume_from=('latest' if CONFIG.get("resume_dataset_generation", True) else None),
        save_all_thetas=True,
        base_seed=CONFIG.get("seed", 42)
    )

    train_time = time.time() - t_start
    print(f"✔ Training dataset generated in {train_time:.1f}s")
    print_dataset_stats(theta_train, T_train, k_train, iv_train, "Training Dataset")

    # Initialize DatasetBuilder
    val_builder = DatasetBuilder(
        process=process,
        device=CONFIG["device"],
        output_dir=CONFIG["output_dir"],
        dataset_type='val'
    )
    # --- VALIDATION DATASET with Random Smiles ---
    print(f"\n>>> Generating Validation Dataset with Random Smiles")
    print(f"    Smiles: {CONFIG['val_smiles']}")
    print(f"    Strikes per smile: {CONFIG['val_strikes']}")
    print(f"    Total points: {CONFIG['val_smiles'] * CONFIG['val_strikes']:,}")

    t_start = time.time()

    theta_val, T_val, k_val, iv_val = val_builder.build_random_smiles_dataset(
        n_smiles=CONFIG["val_smiles"],
        n_strikes_per_smile=CONFIG["val_strikes"],
        n_paths=CONFIG["mc_paths"],
        spot=CONFIG["spot"],
        normalize=True,
        compute_stats_from=train_builder,  # Use training statistics
        show_progress=True,
        checkpoint_every=CONFIG.get("dataset_checkpoint_every", 100),
        resume_from=('latest' if CONFIG.get("resume_dataset_generation", True) else None),
        base_seed=CONFIG.get("seed", 42)  # for consistency with training
    )

    val_time = time.time() - t_start
    print(f"✔ Validation dataset generated in {val_time:.1f}s")
    print_dataset_stats(theta_val, T_val, k_val, iv_val, "Validation Dataset")

    # Save datasets
    print(f"\n>>> Saving datasets to {CONFIG['output_dir']}/datasets/")

    torch.save({
        'theta': theta_train, 'T': T_train, 'k': k_train, 'iv': iv_train,
        'theta_mean': train_builder.theta_mean, 'theta_std': train_builder.theta_std,
        'T_mean': train_builder.T_mean, 'T_std': train_builder.T_std,
        'k_mean': train_builder.k_mean, 'k_std': train_builder.k_std,
        'iv_mean': train_builder.iv_mean, 'iv_std': train_builder.iv_std,
        'config': CONFIG
    }, f"{CONFIG['output_dir']}/datasets/train_random_grids.pt")

    torch.save({
        'theta': theta_val, 'T': T_val, 'k': k_val, 'iv': iv_val
    }, f"{CONFIG['output_dir']}/datasets/val_random_smiles.pt")

    print("✔ Datasets saved")

    # Visualize some examples
    visualize_random_grids_sample(train_builder, theta_train, T_train, k_train, iv_train, n_samples=2)

    # Cleanup dataset checkpoints (optional via CONFIG)
    if CONFIG.get("cleanup_checkpoints_after_completion", True):
        cleanup_dataset_checkpoints(f"{CONFIG['output_dir']}/checkpoints/random_grids")

    return train_builder, (theta_train, T_train, k_train, iv_train), (theta_val, T_val, k_val, iv_val)

In [7]:
# ===============================================================
# === CELL 6: POINTWISE NETWORK TRAINING ===
# ===============================================================

def train_pointwise_network(builder, train_data, val_data):
    """PointwiseNetworkPricer training"""

    print("\n" + "="*60)
    print("POINTWISE NETWORK TRAINING")
    print("="*60)

    theta_train, T_train, k_train, iv_train = train_data
    theta_val, T_val, k_val, iv_val = val_data

    # Create the pricer
    pricer = PointwiseNetworkPricer(
        process=builder.process,
        hidden_layers=CONFIG["hidden_layers"],
        activation=CONFIG["activation"],
        device=CONFIG["device"],
        enable_smile_repair=False  # Not necessary for pointwise
    )

    print(f"\n✔ Created PointwiseNetworkPricer")
    print(f"  Architecture: {builder.process.num_params} + 2 → {CONFIG['hidden_layers']} → 1")
    print(f"  Activation: {CONFIG['activation']}")
    print(f"  Total parameters: {sum(p.numel() for p in pricer.net.parameters()):,}")

    # Set normalization statistics
    pricer.set_normalization_stats(
        builder.theta_mean,
        builder.theta_std,
        builder.iv_mean,
        builder.iv_std
    )
    pricer.set_pointwise_normalization_stats(
        builder.T_mean,
        builder.T_std,
        builder.k_mean,
        builder.k_std
    )

    # Setup training
    use_cuda = str(CONFIG["device"]).startswith("cuda")
    num_workers = 2 if use_cuda else 0
    train_loader = DataLoader(
        TensorDataset(theta_train, T_train, k_train, iv_train),
        batch_size=CONFIG["batch_size"],
        shuffle=True,
        num_workers=0,  # ALWAYS 0 to avoid CUDA issues
        pin_memory=False
    )

    val_loader = DataLoader(
        TensorDataset(theta_val, T_val, k_val, iv_val),
        batch_size=CONFIG["batch_size"],
        shuffle=False,
        num_workers=0,  # ALWAYS 0
        pin_memory=False
    )

    optimizer = torch.optim.Adam(pricer.net.parameters(), lr=CONFIG["learning_rate"])
    loss_fn = nn.MSELoss()

    # Learning rate scheduler
    if CONFIG["lr_scheduler"]:
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode='min', factor=0.5, patience=10
        )

    # ---- Resume / Warm-start --------------------------------------------
    def _extract_model_state(ckpt_obj):
        # 1) raw OrderedDict (state_dict saved directly)
        from collections.abc import Mapping
        if isinstance(ckpt_obj, Mapping):
            # prefer known keys
            for key in ("model_state_dict", "state_dict", "net_state_dict", "pricer_state_dict"):
                sd = ckpt_obj.get(key, None)
                if isinstance(sd, Mapping):
                    return sd
            # if all fail, perhaps the entire ckpt is already a state_dict
            if all(hasattr(v, "shape") for v in ckpt_obj.values()):
                return ckpt_obj  # raw state_dict
        return None

    def _strip_module_prefix(sd):
        if any(k.startswith("module.") for k in sd.keys()):
            return {k.replace("module.", "", 1): v for k, v in sd.items()}
        return sd
    start_epoch = 1
    train_losses = []
    val_losses = []
    best_val_loss = float('inf')
    best_weights = None
    best_epoch = 0

    if CONFIG.get("resume_training", False):
        import glob, re, os
        ckpt_path = CONFIG.get("resume_from")
        if ckpt_path in (None, "latest"):
            ckpt_dir = f"{CONFIG['output_dir']}/models/checkpoints"
            ckpts = glob.glob(os.path.join(ckpt_dir, "checkpoint_epoch_*.pt"))
            if ckpts:
                ckpt_path = max(
                    ckpts,
                    key=lambda p: int(re.search(r"epoch_(\d+)", p).group(1))
                )
        if ckpt_path and os.path.exists(ckpt_path):
            ckpt = torch.load(ckpt_path, map_location=CONFIG["device"])
            model_sd = _extract_model_state(ckpt)
            if model_sd is None:
                raise ValueError(
                    f"Checkpoint '{ckpt_path}' does not contain a model state_dict. "
                    f"Available keys: {list(ckpt.keys()) if isinstance(ckpt, dict) else type(ckpt)}"
                )
            model_sd = _strip_module_prefix(model_sd)
            pricer.net.load_state_dict(model_sd, strict=False)
            opt_sd = ckpt.get("optimizer_state_dict", None) if isinstance(ckpt, dict) else None
            if opt_sd:
                optimizer.load_state_dict(opt_sd)
            else:
                print(">>> Checkpoint has no optimizer state; continuing with fresh optimizer.")
            start_epoch = int(ckpt["epoch"]) + 1
            # retrieve history (if present) for reasonable best_*
            train_losses = ckpt.get("train_losses", [])
            val_losses = ckpt.get("val_losses", [])
            if val_losses:
                best_val_loss = min(val_losses)
                best_epoch = val_losses.index(best_val_loss) + 1
                best_weights = pricer.net.state_dict().copy()
            print(f">>> Resuming from {ckpt_path} (start_epoch={start_epoch})")
    elif CONFIG.get("warm_start", False):
        import os
        wp = CONFIG.get("warm_start_path")
        if not wp:
            # prefer the last best saved by the script
            cand = f"{CONFIG['output_dir']}/models/best_model.pt"
            wp = cand if os.path.exists(cand) else f"{CONFIG['output_dir']}/models/pointwise_random_grids_latest.pt"
        if os.path.exists(wp):
            sd = torch.load(wp, map_location=CONFIG["device"])
            state = sd.get("model_state_dict", sd)
            pricer.net.load_state_dict(state, strict=False)
            best_weights = pricer.net.state_dict().copy()  # starting point
            print(f">>> Warm-start from {wp}")

    # If the checkpoint is already beyond the requested number of epochs, exit cleanly
    if start_epoch > CONFIG["epochs"]:
        # try to derive best
        if val_losses:
            best_val_loss = float(min(val_losses))
            best_epoch    = 1 + val_losses.index(min(val_losses))
        else:
            # fallback: if ckpt had 'val_loss' and 'epoch'
            best_val_loss = float(ckpt.get('val_loss', float('nan'))) if isinstance(ckpt, dict) else float('nan')
            best_epoch    = int(ckpt.get('epoch', start_epoch-1))     if isinstance(ckpt, dict) else start_epoch-1
        history = {
            "train_losses": train_losses,
            "val_losses": val_losses,
            "best_val_loss": best_val_loss,
            "best_epoch": best_epoch,
            "start_epoch": start_epoch,
            "epochs": CONFIG["epochs"],
            "resumed": True,
        }
        return pricer, history


    # Training loop with early stopping
    patience_counter = 0

    print(f"\n>>> Starting training for {CONFIG['epochs']} epochs")
    print(f"    Batch size: {CONFIG['batch_size']}")
    print(f"    Learning rate: {CONFIG['learning_rate']}")

    for epoch in range(start_epoch, CONFIG["epochs"] + 1):
        # Training
        pricer.net.train()
        train_loss = 0.0
        n_train_batches = 0

        for theta_batch, T_batch, k_batch, iv_batch in train_loader:
            # Forward pass
            pred = pricer.forward(theta_batch, T_batch, k_batch)
            loss = loss_fn(pred, iv_batch)

            # Backward pass
            optimizer.zero_grad()
            loss.backward()

            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(pricer.net.parameters(), max_norm=1.0)

            optimizer.step()

            train_loss += loss.item() * len(theta_batch)
            n_train_batches += len(theta_batch)

        train_loss /= n_train_batches
        train_losses.append(train_loss)

        # Validation
        pricer.net.eval()
        val_loss = 0.0
        n_val_batches = 0

        with torch.no_grad():
            for theta_batch, T_batch, k_batch, iv_batch in val_loader:
                pred = pricer.forward(theta_batch, T_batch, k_batch)
                loss = loss_fn(pred, iv_batch)
                val_loss += loss.item() * len(theta_batch)
                n_val_batches += len(theta_batch)

        val_loss /= n_val_batches
        val_losses.append(val_loss)

        # Learning rate scheduling
        if CONFIG["lr_scheduler"]:
            scheduler.step(val_loss)

        # Check for best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_weights = pricer.net.state_dict().copy()
            best_epoch = epoch
            patience_counter = 0

            # Save best model
            torch.save({
                'model_state_dict': best_weights,
                'epoch': best_epoch,
                'val_loss': best_val_loss,
                'config': CONFIG,
                'normalization_stats': {
                    'theta_mean': builder.theta_mean,
                    'theta_std': builder.theta_std,
                    'T_mean': builder.T_mean,
                    'T_std': builder.T_std,
                    'k_mean': builder.k_mean,
                    'k_std': builder.k_std,
                    'iv_mean': builder.iv_mean,
                    'iv_std': builder.iv_std
                }
            }, f"{CONFIG['output_dir']}/models/best_model.pt")
        else:
            patience_counter += 1

        # Logging
        if epoch % 5 == 0 or epoch == 1:
            current_lr = optimizer.param_groups[0]['lr']
            print(f"Epoch {epoch:3d}/{CONFIG['epochs']} | "
                  f"Train: {train_loss:.6f} | Val: {val_loss:.6f} | "
                  f"Best: {best_val_loss:.6f} (ep {best_epoch}) | "
                  f"LR: {current_lr:.1e}")

        # Periodic checkpoint
        # Default/fallback if missing in CONFIG
        checkpoint_every = CONFIG.get("checkpoint_every", 100)
        if checkpoint_every and (epoch % checkpoint_every == 0):
            checkpoint = {
                'epoch': epoch,
                'model_state_dict': pricer.net.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'train_loss': train_loss,
                'val_loss': val_loss,
                'train_losses': train_losses,
                'val_losses': val_losses
            }
            torch.save(checkpoint,
                      f"{CONFIG['output_dir']}/models/checkpoints/checkpoint_epoch_{epoch}.pt")

        # Early stopping
        if patience_counter >= CONFIG["early_stopping_patience"]:
            print(f"\n>>> Early stopping triggered at epoch {epoch}")
            break

        # Clear cache periodically
        if epoch % 10 == 0:
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
            gc.collect()

    # Load best weights
    pricer.net.load_state_dict(best_weights)
    print(f"\n✔ Training completed. Best model from epoch {best_epoch} (Val Loss: {best_val_loss:.6f})")

    # Save training history
    history = {
        'train_losses': train_losses,
        'val_losses': val_losses,
        'best_epoch': best_epoch,
        'best_val_loss': best_val_loss,
        'config': CONFIG
    }

    with open(f"{CONFIG['output_dir']}/logs/training_history.pkl", 'wb') as f:
        pickle.dump(history, f)

    # Plot training curves
    plot_training_history(train_losses, val_losses,
                         f"{CONFIG['output_dir']}/visualizations/training_curves.png")

    return pricer, history

In [8]:
# ===============================================================
# === CELL 7: TESTING AND INFERENCE ===
# ===============================================================

def _training_k_bounds_for_T(builder, T, spot: float = 1.0):
    """
    Returns (k_min, k_max) of the training domain for a given T.
    Uses the same parameters as the DatasetBuilder: K_min = S(1 - l√T), K_max = S(1 + u√T)
    with numerical clamps [0.05S, 3S], then k = log(K/S).
    """
    sp = getattr(builder, "_strike_params", None) or {"l": 0.55, "u": 0.30}
    l, u = float(sp["l"]), float(sp["u"])
    T_t = torch.as_tensor(T, dtype=torch.float32, device=builder.device)
    sqrtT = torch.sqrt(T_t)
    K_min = torch.clamp(torch.tensor(spot, device=builder.device) * (1.0 - l * sqrtT), min=0.05 * spot)
    K_max = torch.clamp(torch.tensor(spot, device=builder.device) * (1.0 + u * sqrtT), max=3.0 * spot)
    k_min = torch.log(K_min / spot).item()
    k_max = torch.log(K_max / spot).item()
    return k_min, k_max

def make_in_domain_logk_grid(builder, maturities: torch.Tensor, n_points: int = 11,
                             safety_eps: float = 1e-3, spot: float = 1.0) -> torch.Tensor:
    """
    Grid of k that is within the INTERSECTION of the training domains for all passed Ts.
    Avoids extrapolation in tests/heatmaps.
    """
    kmins, kmaxs = [], []
    for T in maturities:
        lo, hi = _training_k_bounds_for_T(builder, T, spot=spot)
        kmins.append(lo); kmaxs.append(hi)
    k_lo = max(kmins) + safety_eps
    k_hi = min(kmaxs) - safety_eps
    if not (k_lo < k_hi):
        raise ValueError("No overlap of training domains between the provided maturities.")
    return torch.linspace(k_lo, k_hi, n_points, device=builder.device, dtype=torch.float32)

def test_and_inference(pricer, builder, val_data):
    """Model testing and inference examples"""

    print("\n" + "="*60)
    print("MODEL TESTING AND INFERENCE")
    print("="*60)

    theta_val, T_val, k_val, iv_val = val_data

    # Test on a subset of the validation set
    n_test = min(1000, len(theta_val))
    test_idx = torch.randperm(len(theta_val))[:n_test]

    theta_test = theta_val[test_idx]
    T_test = T_val[test_idx]
    k_test = k_val[test_idx]
    iv_test = iv_val[test_idx]

    # Calculate metrics
    mae, rmse, mape = test_predictions(pricer, builder,
                                       theta_test, T_test, k_test, iv_test,
                                       n_samples=5)

    # --- MC vs NN Comparison ---
    print("\n>>> Generating MC vs NN Comparison")

    # Select random parameters for comparison
    test_theta_mc = builder.sample_theta_lhs(1)[0]

    # Define grid for comparison
    test_maturities = torch.tensor([0.08, 0.17, 0.25, 0.5, 1.0], device=CONFIG["device"])
    # test_logK = torch.linspace(-0.3, 0.3, 11, device=CONFIG["device"]) # Fixed grid
    test_logK = make_in_domain_logk_grid(builder, test_maturities, n_points=11)


    # Generate comparison
    iv_mc, iv_nn = visualize_mc_vs_nn_comparison(
        builder, pricer, test_theta_mc,
        test_maturities, test_logK,
        n_mc_paths=50000,
        save_path=f"{CONFIG['output_dir']}/visualizations/mc_vs_nn_comparison.png"
    )

    # --- Inference test on new points ---
    print("\n>>> Testing inference on new random points")

    # Generate some random parameters
    test_params = builder.sample_theta_lhs(3)
    print(f"\nTest parameters (denormalized):")
    for i, params in enumerate(test_params):
        param_str = ", ".join([f"{p:.3f}" for p in params.cpu().numpy()])
        print(f"  Sample {i+1}: [{param_str}]")

    # Generate random points (T, k)
    test_T = torch.tensor([0.1, 0.25, 0.5, 1.0, 2.0], device=CONFIG["device"])
    test_k = torch.tensor([-0.2, -0.1, 0.0, 0.1, 0.2], device=CONFIG["device"])

    # Create full grid
    test_theta_grid = test_params[0].unsqueeze(0).repeat(len(test_T) * len(test_k), 1)
    T_grid, k_grid = torch.meshgrid(test_T, test_k, indexing='ij')
    T_flat = T_grid.flatten()
    k_flat = k_grid.flatten()

    # Prediction
    with torch.no_grad():
        iv_pred = pricer.price_iv(test_theta_grid, T_flat, k_flat, denormalize_output=True)

    # Visualize predicted surface
    iv_surface = iv_pred.reshape(len(test_T), len(test_k))

    fig = plt.figure(figsize=(12, 5))

    # 3D surface plot
    ax1 = fig.add_subplot(121, projection='3d')
    T_mesh, k_mesh = np.meshgrid(test_T.cpu(), test_k.cpu(), indexing='ij')
    surf = ax1.plot_surface(k_mesh, T_mesh, iv_surface.cpu().numpy(),
                           cmap='viridis', alpha=0.9)
    ax1.set_xlabel('Log-Moneyness')
    ax1.set_ylabel('Maturity')
    ax1.set_zlabel('Implied Volatility')
    ax1.set_title('Predicted IV Surface')
    fig.colorbar(surf, ax=ax1, shrink=0.5)

    # Heatmap
    ax2 = fig.add_subplot(122)
    im = ax2.imshow(iv_surface.cpu().numpy(), aspect='auto', cmap='viridis',
                    extent=[test_k.min().item(), test_k.max().item(),
                           test_T.min().item(), test_T.max().item()])
    ax2.set_xlabel('Log-Moneyness')
    ax2.set_ylabel('Maturity')
    ax2.set_title('IV Surface Heatmap')
    fig.colorbar(im, ax=ax2)

    plt.tight_layout()
    plt.savefig(f"{CONFIG['output_dir']}/visualizations/inference_surface.png", dpi=150)
    plt.show()

    print(f"\n✔ Inference test completed")
    print(f"  Surface shape: {iv_surface.shape}")
    print(f"  IV range: [{iv_surface.min():.4f}, {iv_surface.max():.4f}]")
    print(f"  Mean IV: {iv_surface.mean():.4f} ± {iv_surface.std():.4f}")

    return mae, rmse, mape

In [9]:
# ===============================================================
# === CELL 8: SAVE & LOAD UTILITIES ===
# ===============================================================

def save_final_model(pricer, builder, history):
    """Saves the final model with all metadata"""

    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    process_name = builder.process.__class__.__name__.lower()

    final_path = f"{CONFIG['output_dir']}/models/final"
    model_name = f"{process_name}_pointwise_random_grids_{timestamp}"

    # Save the complete model
    model_dict = {
        'model_state_dict': pricer.net.state_dict(),
        'process_name': builder.process.__class__.__name__,
        'process_key': CONFIG['process'],
        'process_params': {
            'num_params': builder.process.num_params,
            'param_names': builder.process.param_info.names if hasattr(builder.process, 'param_info') else [],
            'param_bounds': builder.process.param_info.bounds if hasattr(builder.process, 'param_info') else [],
        },
        'network_config': {
            'hidden_layers': CONFIG['hidden_layers'],
            'activation': CONFIG['activation'],
        },
        'normalization_stats': {
            'theta_mean': builder.theta_mean.cpu(),
            'theta_std': builder.theta_std.cpu(),
            'T_mean': builder.T_mean.cpu(),
            'T_std': builder.T_std.cpu(),
            'k_mean': builder.k_mean.cpu(),
            'k_std': builder.k_std.cpu(),
            'iv_mean': builder.iv_mean.cpu(),
            'iv_std': builder.iv_std.cpu(),
        },
        'training_config': CONFIG,
        'training_history': history,
        'timestamp': timestamp
    }

    # Save as .pt
    torch.save(model_dict, f"{final_path}/{model_name}.pt")

    # extract fields robustly
    train_losses = history.get('train_losses', [])
    val_losses   = history.get('val_losses', [])
    if 'best_val_loss' in history and 'best_epoch' in history:
        best_val_loss = float(history['best_val_loss'])
        best_epoch    = int(history['best_epoch'])
    elif val_losses:
        best_val_loss = float(min(val_losses))
        best_epoch    = 1 + val_losses.index(min(val_losses))
    else:
        best_val_loss = float('nan')
        best_epoch    = -1


    # Also save configuration as JSON for reference
    config_dict = {
        'model_name': model_name,
        'process_name': builder.process.__class__.__name__,
        'process_key': CONFIG['process'],
        'network_architecture': f"{builder.process.num_params}+2 → {CONFIG['hidden_layers']} → 1",
        'training_config': CONFIG,
        'best_val_loss': best_val_loss,
        'best_epoch': best_epoch,
        'total_epochs': len(train_losses),
        'timestamp': timestamp
    }


    with open(f"{final_path}/{model_name}_config.json", 'w') as f:
        json.dump(config_dict, f, indent=2)

    # Copy as "latest" for easy access
    import shutil
    shutil.copy2(f"{final_path}/{model_name}.pt",
                 f"{final_path}/pointwise_random_grids_latest.pt")
    shutil.copy2(f"{final_path}/{model_name}_config.json",
                 f"{final_path}/pointwise_random_grids_latest_config.json")

    print(f"\n✔ Model saved to Google Drive:")
    print(f"  Directory: {final_path}")
    print(f"  Model: {model_name}.pt")
    print(f"  Config: {model_name}_config.json")

    return model_name


def load_saved_model(model_path, device='cpu'):
    """Loads a saved model"""

    print(f"\nLoading model from: {model_path}")

    # Load the checkpoint
    checkpoint = torch.load(model_path, map_location=device)

    # Recreate the process from the saved key (fallback if absent)
    process_key = checkpoint.get('process_key')
    if process_key is None:
        name = str(checkpoint.get('process_name', '')).lower()
        if 'rough' in name and 'bergomi' in name:
            process_key = 'rough_bergomi'
        elif 'rough' in name and 'heston' in name:
            process_key = 'rough_heston'
        elif 'heston' in name:
            process_key = 'heston'
        else:
            raise ValueError(f"Unrecognized process key from '{name}'")
    process = ProcessFactory.create(process_key)

    # Recreate the pricer
    loaded_pricer = PointwiseNetworkPricer(
        process=process,
        hidden_layers=checkpoint['network_config']['hidden_layers'],
        activation=checkpoint['network_config']['activation'],
        device=device
    )

    # Load weights
    loaded_pricer.net.load_state_dict(checkpoint['model_state_dict'])

    # Set normalization statistics
    norm_stats = checkpoint['normalization_stats']
    loaded_pricer.set_normalization_stats(
        norm_stats['theta_mean'].to(device),
        norm_stats['theta_std'].to(device),
        norm_stats['iv_mean'].to(device),
        norm_stats['iv_std'].to(device)
    )
    loaded_pricer.set_pointwise_normalization_stats(
        norm_stats['T_mean'].to(device),
        norm_stats['T_std'].to(device),
        norm_stats['k_mean'].to(device),
        norm_stats['k_std'].to(device)
    )


    loaded_pricer.eval()

    print(f"✔ Model loaded successfully")
    print(f"  Process: {process_key} ({checkpoint.get('process_name')})")
    print(f"  Architecture: {checkpoint['network_config']['hidden_layers']}")
    print(f"  Best val loss: {checkpoint['training_history']['best_val_loss']:.6f}")


    return loaded_pricer, checkpoint

In [10]:
# ===============================================================
# === CELL 9: MAIN FUNCTION ===
# ===============================================================

def run_complete_random_grids_training():
    """Runs the complete training with Random Grids"""

    print("\n" + "="*60)
    print(" RANDOM GRIDS POINTWISE NETWORK TRAINING ")
    print("="*60)
    print(f"\nConfiguration:")
    print(f"  Process: {CONFIG['process']}")
    print(f"  Device: {CONFIG['device']}")
    print(f"  Training surfaces: {CONFIG['train_surfaces']}")
    print(f"  Validation smiles: {CONFIG['val_smiles']}")
    print(f"  Network: {CONFIG['hidden_layers']}")
    print(f"  Output: {CONFIG['output_dir']}")

    # Step 1: Generate datasets
    print("\n" + "="*60)
    print("STEP 1: DATASET GENERATION")
    print("="*60)

    builder, train_data, val_data = generate_datasets()

    # Cleanup memory after dataset generation
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()

    # Step 2: Training
    print("\n" + "="*60)
    print("STEP 2: NETWORK TRAINING")
    print("="*60)

    pricer, history = train_pointwise_network(builder, train_data, val_data)

    # Step 3: Testing
    print("\n" + "="*60)
    print("STEP 3: MODEL TESTING")
    print("="*60)

    mae, rmse, mape = test_and_inference(pricer, builder, val_data)

    # Step 3b: MC vs NN Detailed Comparison
    print("\n" + "="*60)
    print("STEP 3b: MC VS NN COMPARISON")
    print("="*60)

    # Generate detailed comparison
    comparison_theta = builder.sample_theta_lhs(1)[0]
    comparison_maturities = torch.tensor([0.08, 0.17, 0.25, 0.5, 1.0, 2.0], device=CONFIG["device"])
    # comparison_logK = torch.linspace(-0.3, 0.3, 13, device=CONFIG["device"]) # Fixed grid
    comparison_logK = make_in_domain_logk_grid(builder, comparison_maturities, n_points=13)

    print(f"\nGenerating detailed comparison with:")
    print(f"  Maturities: {comparison_maturities.cpu().numpy()}")
    print(f"  Strikes: {len(comparison_logK)} points from {comparison_logK.min():.2f} to {comparison_logK.max():.2f}")

    iv_mc_ref, iv_nn_pred = visualize_mc_vs_nn_comparison(
        builder, pricer, comparison_theta,
        comparison_maturities, comparison_logK,
        n_mc_paths=100000,  # More paths for accurate reference
        save_path=f"{CONFIG['output_dir']}/visualizations/smile_comparison_detailed.png"
    )

    # Add heatmap comparison as well
    visualize_surface_heatmap_comparison(
        iv_mc_ref, iv_nn_pred,
        comparison_maturities, comparison_logK,
        save_path=f"{CONFIG['output_dir']}/visualizations/surface_comparison_heatmap.png"
    )


    # Step 4: Save model
    print("\n" + "="*60)
    print("STEP 4: SAVING MODEL")
    print("="*60)

    model_name = save_final_model(pricer, builder, history)

    # Step 5: Verify loading
    print("\n" + "="*60)
    print("STEP 5: TESTING MODEL LOADING")
    print("="*60)

    model_path = f"{CONFIG['output_dir']}/models/final/pointwise_random_grids_latest.pt"
    loaded_pricer, loaded_checkpoint = load_saved_model(model_path, device=CONFIG['device'])

    # Test that the loaded model gives the same results
    print("\n>>> Verifying loaded model...")
    test_theta = torch.randn(5, builder.process.num_params, device=CONFIG['device'])
    test_T = torch.rand(5, device=CONFIG['device']) * 2
    test_k = torch.randn(5, device=CONFIG['device']) * 0.3


    pricer.eval()
    with torch.no_grad():
        original_pred = pricer.price_iv(test_theta, test_T, test_k, denormalize_output=True)
        loaded_pred = loaded_pricer.price_iv(test_theta, test_T, test_k, denormalize_output=True)

        diff = (original_pred - loaded_pred).abs().max()
        print(f"  Max difference: {diff:.8f}")

        if diff < 1e-6:
            print("  ✔ Loaded model produces identical results")
        else:
            print("  ⚠ Warning: Loaded model produces different results")

    # Final Summary
    print("\n" + "="*60)
    print("TRAINING COMPLETED SUCCESSFULLY")
    print("="*60)
    print(f"\nFinal Results:")
    print(f"  Best Validation Loss: {history['best_val_loss']:.6f}")
    print(f"  Best Epoch: {history['best_epoch']}/{len(history['train_losses'])}")
    print(f"  Test MAE: {mae:.6f}")
    print(f"  Test RMSE: {rmse:.6f}")
    print(f"  Test MAPE: {mape:.2f}%")
    print(f"\nModel saved as: {model_name}")
    print(f"Location: {CONFIG['output_dir']}/models/final/")

    return pricer, builder, history

In [11]:
# ===============================================================
# === CELL 10: UTILITY FOR SURFACE GENERATION ===
# ===============================================================

def generate_full_surface(pricer, builder, theta, T_range=(0.01, 3.0), k_range=(-0.5, 0.5), n_points=50):
    """
    Generates a complete IV surface for a given set of parameters

    Args:
        pricer: The trained model
        builder: The DatasetBuilder for denormalization
        theta: Model parameters (not normalized)
        T_range: Range of maturities
        k_range: Range of log-moneyness
        n_points: Number of points per dimension
    """

    # Create grid
    T_grid = torch.linspace(T_range[0], T_range[1], n_points, device=CONFIG['device'])
    k_grid = torch.linspace(k_range[0], k_range[1], n_points, device=CONFIG['device'])

    # Meshgrid
    T_mesh, k_mesh = torch.meshgrid(T_grid, k_grid, indexing='ij')
    T_flat = T_mesh.flatten()
    k_flat = k_mesh.flatten()

    # Expand theta for all points
    theta_expanded = theta.unsqueeze(0).repeat(len(T_flat), 1)

    # Prediction
    pricer.eval()
    with torch.no_grad():
        iv_flat = pricer.price_iv(theta_expanded, T_flat, k_flat, denormalize_output=True)

    # Reshape to surface
    iv_surface = iv_flat.reshape(n_points, n_points)

    # Visualize
    fig = plt.figure(figsize=(15, 5))

    # 3D plot
    ax1 = fig.add_subplot(131, projection='3d')
    surf = ax1.plot_surface(k_mesh.cpu(), T_mesh.cpu(), iv_surface.cpu(),
                           cmap='viridis', alpha=0.9)
    ax1.set_xlabel('Log-Moneyness')
    ax1.set_ylabel('Maturity (years)')
    ax1.set_zlabel('Implied Volatility')
    ax1.set_title('IV Surface - 3D View')
    ax1.view_init(elev=20, azim=45)
    fig.colorbar(surf, ax=ax1, shrink=0.5)

    # Heatmap
    ax2 = fig.add_subplot(132)
    im = ax2.imshow(iv_surface.cpu(), aspect='auto', cmap='viridis',
                   extent=[k_range[0], k_range[1], T_range[0], T_range[1]],
                   origin='lower')
    ax2.set_xlabel('Log-Moneyness')
    ax2.set_ylabel('Maturity (years)')
    ax2.set_title('IV Surface - Heatmap')
    fig.colorbar(im, ax=ax2)

    # Smile slices at different maturities
    ax3 = fig.add_subplot(133)
    maturity_indices = [0, n_points//4, n_points//2, 3*n_points//4, n_points-1]
    for idx in maturity_indices:
        T_val = T_grid[idx].item()
        ax3.plot(k_grid.cpu(), iv_surface[idx, :].cpu(),
                label=f'T={T_val:.2f}y', linewidth=2)
    ax3.set_xlabel('Log-Moneyness')
    ax3.set_ylabel('Implied Volatility')
    ax3.set_title('Volatility Smiles')
    ax3.legend()
    ax3.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig(f"{CONFIG['output_dir']}/visualizations/full_surface.png", dpi=150)
    plt.show()

    # Print statistics
    print(f"\nSurface Statistics:")
    print(f"  Shape: {iv_surface.shape}")
    print(f"  IV range: [{iv_surface.min():.4f}, {iv_surface.max():.4f}]")
    print(f"  Mean IV: {iv_surface.mean():.4f} ± {iv_surface.std():.4f}")

    # Parameters used (denormalized)
    param_names = builder.process.param_info.names if hasattr(builder.process, 'param_info') else ['p'+str(i) for i in range(len(theta))]
    print(f"\nParameters used:")
    for name, value in zip(param_names, theta.cpu().numpy()):
        print(f"  {name}: {value:.4f}")

    return iv_surface, T_mesh, k_mesh

In [12]:
# ===============================================================
# === MAIN EXECUTION ===
# ===============================================================

# Run the complete training
if __name__ == "__main__":
    # Run training
    pricer, builder, history = run_complete_random_grids_training()

    # Generate a complete surface as a final example
    print("\n" + "="*60)
    print("GENERATING FULL SURFACE EXAMPLE")
    print("="*60)

    # Use example parameters
    example_theta = builder.sample_theta_lhs(1)[0]
    iv_surface, T_mesh, k_mesh = generate_full_surface(
        pricer, builder, example_theta,
        T_range=(0.05, 2.0),
        k_range=(-0.4, 0.4),
        n_points=40
    )

    print("\n✓ COMPLETE EXECUTION FINISHED!")
    print(f"All results saved to: {CONFIG['output_dir']}")


 RANDOM GRIDS POINTWISE NETWORK TRAINING 

Configuration:
  Process: rough_heston
  Device: cuda
  Training surfaces: 7000
  Validation smiles: 10000
  Network: [30, 30, 30, 30]
  Output: /content/drive/MyDrive/random_grids_results

STEP 1: DATASET GENERATION

DATASET GENERATION WITH RANDOM GRIDS

✔ Created process: RoughHestonProcess
  Parameters: ['H', 'nu', 'rho', 'kappa', 'theta_var']
  Supports absorption: True

>>> Generating Training Dataset with Random Grids
    Surfaces: 7000
    Maturities per surface: 11
    Strikes per maturity: 13
    Total points: 1,001,000

Building Random Grids Pointwise Dataset: 7000 × 11 × 13
Starting from surface 0 [train]

Building Random Grids Pointwise Dataset: 7000 × 11 × 13
Starting from surface 0


Generating random grids:   0%|          | 3/7000 [01:53<73:30:20, 37.82s/it]


KeyboardInterrupt: 